<a href="https://colab.research.google.com/github/dcolinmorgan/test/blob/master/LTRC_TS_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LTCOPD dataset analyzed for time series prediction

ran panda nets for control and case [HERE in "run large PANDA"](https://colab.research.google.com/drive/1ER3FudwHf62xpVaqtA55XevX--E4INUe?authuser=1#scrollTo=3NiZai_nLIts&uniqifier=2)

In [1]:
import os,glob,gc
import pandas as pd
import numpy as np
from psutil import *
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from scipy.stats import zscore
import tensorflow as tf


In [2]:
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.offsets import DateOffset
import math
from sklearn.metrics import mean_squared_error

import keras
from keras import Sequential
from keras.layers import Dense, LSTM, Conv1D, Bidirectional, Lambda,Dropout

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
links=pd.read_csv('drive/My Drive/LARGE_PANDA/LARGECOPD_links.txt',names=['TF','gene'])
CASE=np.load('drive/My Drive/LARGE_PANDA/LTRCcase.npy')
CASE_ages=pd.read_csv('drive/My Drive/LARGE_PANDA/LTRCcase_ages.txt',names=['ages'])

In [4]:
CASE=pd.DataFrame(CASE)
CASE.columns=CASE_ages[0:]['ages']
# CASE.index=links.TF+'_'+links.gene

In [6]:
# links=pd.read_csv('drive/My Drive/LARGE_PANDA/LARGECOPD_links.txt',names=['TF','gene'])
# CONTROL=np.load('drive/My Drive/LARGE_PANDA/LTRCcontrol.npy')
# CONTROL_ages=pd.read_csv('drive/My Drive/LARGE_PANDA/LTRCcontrol_ages.txt',names=['ages'])

In [7]:
# CONTROL=pd.DataFrame(CONTROL)
# CONTROL.columns=CONTROL_ages[0:]['ages']
# CONTROL.index=links.TF+'_'+links.gene

# FORMAT DATA

## CASE

In [ ]:
tdata=pd.DataFrame(CASE.T, index=CASE.columns)#.interpolate(method='time')
tdata['Datetime'] = pd.to_datetime('19'+(CASE.columns.astype(str)) + ' ' +'10:00:00 ')
del CASE
CASE=pd.DataFrame()
tdata = tdata.set_index('Datetime')
# ALL.iloc[1,:].to_list()#.dropna(how='any')
tdata=tdata.interpolate(method='time')


##CONTROL

In [ ]:
# tdata=pd.DataFrame(CASE.T, index=CASE.columns)#.interpolate(method='time')
# tdata['Datetime'] = pd.to_datetime('19'+(CASE.columns.astype(str)) + ' ' +'10:00:00 ')
# tdata = tdata.set_index('Datetime')
# # ALL.iloc[1,:].to_list()#.dropna(how='any')
# tdata=tdata.interpolate(method='time')

In [ ]:
split_time = 30
heat=pd.DataFrame()
window_size = 6
batch_size = 10
shuffle_buffer_size = 10

i=int(np.random.randint(1, high=ttdata.shape[0], size=1) )
series=np.array(ttdata.iloc[i,:])

time_train = ALL.columns[:split_time]
x_train = series[:split_time]
time_valid = ALL.columns[split_time:]
x_valid = series[split_time:]
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)


In [ ]:
dataset_x = np.array(tdata)#np.concatenate((dataset_delta_DJI, dataset_delta_APPL, dataset_delta_AMAZN), axis = 1)
# dataset_y = dataset_delta_DJI
# normalize the dataset
scaler_multi = MinMaxScaler(feature_range=(0, 1))
scaler_multi.fit_transform(dataset_x.reshape(-1, 1))
dataset_x = scaler_multi.transform(dataset_x)
# dataset_y = scaler_multi.transform(dataset_y)
# split into train and test sets
# dataset_x
train_size = int(len(dataset_x) * 0.67)
test_size = len(dataset_x) - train_size
train_x, test_x = dataset_x[0:train_size,:], dataset_x[train_size:len(dataset_x),:]

# train_x=train_x.reshape(1,train_x.shape[0],train_x.shape[1])

# train_y, test_y = dataset_y[0:train_size,:], dataset_y[train_size:len(dataset_y),:]

In [ ]:
# dataset_xx=pd.DataFrame(dataset_x)
# dataset_yy=pd.DataFrame(0,index=tdata.index[0:5],columns=tdata.columns)
# dataset_xx.append(dataset_yy)
n_input=6
tdata=tdata[::-1]
add_dates = [tdata.index[-1] + DateOffset(years=x) for x in range(0,n_input+1) ]
future_dates = pd.DataFrame(0,index=add_dates[1:],columns=tdata.columns)

t2data=tdata.append(future_dates)
tdata=tdata[::-1]
t2data=t2data[::-1]
t2data=np.array(t2data)

scaler_multi = MinMaxScaler(feature_range=(0, 1)) ## batch normalization between layers
scaler_multi.fit_transform(t2data.reshape(-1, 1))
t2data = scaler_multi.transform(t2data)

# tdata['Datetime'] = pd.to_datetime('19'+(ALL.columns.astype(str)) + ' ' +'10:00:00 ')
# tdata = tdata.set_index('Datetime')
# # ALL.iloc[1,:].to_list()#.dropna(how='any')
# tdata=tdata.interpolate(method='time')

In [ ]:
# look_back = 6
# train_data_gen = TimeseriesGenerator(train_x, train_x,
#                                length=look_back, sampling_rate=1,stride=1,
#                                batch_size=6)
# test_data_gen = TimeseriesGenerator(test_x, test_x,
#                                length=look_back, sampling_rate=1,stride=1,
#                                batch_size=1)

# pred_data_gen = TimeseriesGenerator(t2data[::-1], t2data[::-1],
#                                length=look_back, sampling_rate=1,stride=1,
#                                batch_size=1)


train = t2data[::-1]
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
n_input = 5
lookback=n_input
n_features = t2data.data.shape[1]
generator = TimeseriesGenerator(train,train,sampling_rate=1, length=lookback)#, batch_size=6)


train_x=train[:-12]
train_y=train[-12:-6]
  ### subset

trainA=train[:-6] 
# trainA=trainA[:,np.var(trainA,axis=0)<.05]
trainA=trainA[:,np.var(trainA,axis=0)<.0001]
train_x=trainA[:-6]
train_y=trainA[-6:]

training_sequence = TimeseriesGenerator(train_x, train_x, length=lookback,batch_size=1)
validation_sequence = TimeseriesGenerator(train_y, train_y,  length=lookback,batch_size=1)

# look_back = 1 
# train_x, train_y = create_data_set(train, look_back) 
# # test_x, test_y = create_data_set(test, look_back) 
# train_x = np.reshape(train_x,
#                         (train_x.shape[0], 1, train_x.shape[1])) 
# # test_x = np.reshape(test_x, 
# #                        (test_x.shape[0], 1, test_x.shape[1]))


In [ ]:
model = Sequential()
## graph neural network then (lower D space) to LSTM (neighborhood) NOT conv1d (spatial from images)
initializer=tf.keras.initializers.Orthogonal()
model.add(LSTM(256, input_shape=(lookback, training_sequence.data.shape[1]),return_sequences=True, kernel_initializer=initializer)) #orthoganal initialization for weight, he  # 5 time steps and 32831 features/genes
model.add(Bidirectional(LSTM(128,activation='relu',return_sequences=True))),
model.add((LSTM(128,activation='relu',return_sequences=True))),
model.add((LSTM(64,activation='relu',return_sequences=True))),
model.add((LSTM(48,activation='relu',return_sequences=True))),
model.add((LSTM(32,activation='relu'))),
# model.add(Dropout(0.15))
# model.add((LSTM(32,activation='relu'))),
model.add(Dense(training_sequence.data.shape[1]))
model.compile(loss='mse', optimizer='adam',metrics=['mse','accuracy'])
print(model.summary())

# generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)
# # define model
# model = Sequential()
# model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))   # 2 x 3
# model.add(Dense(3))
# model.compile(optimizer='adam', loss='mse')
# # fit model
# model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=0)
# # make a one step prediction out of sample
# x_input = array([[90, 95], [100, 105],[12,21]]).reshape((1, n_input, n_features))

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)
from IPython.display import Image
Image(filename='model.png')

In [ ]:
from keras.callbacks import EarlyStopping, LearningRateScheduler,ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 20)),
    EarlyStopping(patience=5, verbose=1),#monitor='loss'),
    # ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=False)
]

In [ ]:
history =model.fit(training_sequence, epochs=100, callbacks=callbacks,validation_data=validation_sequence)


In [ ]:
model.save('drive/MyDrive/LARGE_PANDA/model_output/CASE.h')
# model.save('drive/MyDrive/LARGE_PANDA/model_output/CONTROL.h')

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# import matplotlib.pyplot as plt

plt.figure(figsize=(18, 8))
plt.plot(hist['epoch'],hist['val_loss'], color= 'green')
# plt.plot(loss, color='red')
# plt.title("Close price of stocks sold")
plt.xlabel("EPOCH")
plt.ylabel("VAL_LOSS")
plt.show()


# load and predict

In [ ]:
from keras.models import load_model
model = load_model('drive/MyDrive/LARGE_PANDA/model_output/CASE.h')

In [ ]:
t2data=t2data[::-1]
train = t2data[:-30]
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
# batch=[]
pred_list = []
# batch = train_y[-n_input:].reshape((1, n_input, t2data.shape[1]))

##subset
train = trainA
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
batch = train_y[-n_input:].reshape((1, n_input, trainA.shape[1]))

# batch=batch.T
for i in range(n_input):
  # print(batch[0,1:,:])
  pred_list.append(model.predict(batch))#[0])  ## comment T then uncomment
  batch = np.append(batch[0,1:,:],pred_list[i],axis=0)
  # print([batch.shape,pred_list[i].T.shape])
  # batch=batch.T
  # print(pred_list[i])
  batch = np.expand_dims(batch, 0)
  # print([i,batch.shape])
print(pd.DataFrame(np.squeeze(pred_list)))

In [ ]:
# pred_list=pd.DataFrame(np.reshape(pred_list,[lookback,t2data.shape[1]]))
pred_list=pd.DataFrame(np.reshape(pred_list,[lookback,trainA.shape[1]]))
# pred_list=scaler.inverse_transform((pred_list))
# scaler = MinMaxScaler()
# scaler.fit(trainA)
# pred_list=scaler.transform((pred_list))
# pred_list = []
# batch = train[-n_input:].reshape((1, n_input, t2data.shape[1]))
# pred_list.append(model.predict(batch.T))
pred_list=(np.squeeze(pred_list))
pred_list=pd.DataFrame(pred_list)
print(pred_list)

# pred=pd.DataFrame(t2data[::-1])[6:]
pred=pd.DataFrame(trainA)
pred=pred.append(pred_list)
pred=pred.reset_index()
del pred['index']

In [ ]:
w=plt.plot(pred.iloc[0:42,0:9])
